# Prédiction du genre d'une personne à partir de sa photo

auteur : Umut Şimşekli & Alexandre Gramfort, challenge SD210 2017

L'objectif est de prédire le genre d'une personne (homme ou femme) à partir de caractéristiques extraites d'une photo.

Les données sont fournies par la société Morpho: http://www.morpho.com/fr

Le fichier que doit fournir chacun est un fichier au format .txt de 8496 lignes::

    1
    0
    -1
    1
    ...

où chaque ligne contient la prédiction. Contrairement à un problème de classification binaire où y=1 ou y=-1, vous avez la possibilité de prédire 0, ce qui signifie que vous ne savez pas. Il y a 8496 images dans l'ensemble de validation.


### Critère de performance 

Vous pouvez donc répondre pour chaque image : homme (y=1), femme (y=-1) ou je-ne-sais-pas (y=0).

Se tromper coûte 10 points et ne pas savoir coûte 1 point. Mathématiquement, le score est calculé de la façon suivante:
 
$score = \frac1{N} \sum_{i=1}^N \Bigl(\mathbb{1}(\hat{y}_i = 0) + 10 \times \mathbb{1}(y_i \hat{y}_i = -1)   \Bigr) $ 

où $\mathbb{1}(\cdot)$ est la fonction indicatrice; $\mathbb{1}(x) = 1$ si $x$ est vrai, et $\mathbb{1}(x) = 0$, sinon.

Plus ce nombre est petit mieux c'est.


# Données d'apprentissage: 

https://www.dropbox.com/s/dqudxed82ljnxa8/training_templates.csv

https://www.dropbox.com/s/l0f9z08rysp0kjy/training_labels.txt


# Données de validation:

https://www.dropbox.com/s/syrry7miykrmjz0/testing_templates.csv

Voyons cela par l'exemple:

In [25]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [26]:
# Critere de performance
def compute_pred_score(y_true, y_pred):
    y_pred_unq =  np.unique(y_pred)
    for i in y_pred_unq:
        if((i != -1) & (i!= 1) & (i!= 0) ):
            raise ValueError('The predictions can contain only -1, 1, or 0!')
    y_comp = y_true * y_pred
    score = float(10*np.sum(y_comp == -1) + np.sum(y_comp == 0))
    score /= y_comp.shape[0]
    return score

In [27]:
X_train_fname = 'training_templates.csv'
y_train_fname = 'training_labels.txt'
X_test_fname  = 'testing_templates.csv'
X_train = pd.read_csv(X_train_fname, sep=',', header=None).values
X_test  = pd.read_csv(X_test_fname,  sep=',', header=None).values
y_train = np.loadtxt(y_train_fname, dtype=np.int)

## Investigation des données

In [28]:
X_train.shape, X_test.shape, y_train.shape

((105600, 128), (8496, 128), (105600,))

In [29]:
print('n_samples : %d, n_features : %d' % X_train.shape)

n_samples : 105600, n_features : 128


In [30]:
np.mean(y_train), np.unique(y_train)

(0.0, array([-1,  1]))

In [31]:
#from sklearn.preprocessing import StandardScaler
#X_scaler = StandardScaler()
#X_train = X_scaler.fit_transform(X_train)
#X_test = X_scaler.transform(X_test)

## Exemple de classification

Voyons maintenant un exemple de classification et de production d'un fichier de soumission

In [32]:
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score

param_grid = {#"hidden_layer_sizes":[(1000,),(1024,),(1100,)],
    #"alpha":np.logspace(-3,-1,5)
    "alpha":np.linspace(0.005,0.015,3),
    "learning_rate_init":[0.001,0.002,0.003,0.004],
}

clf = MLPClassifier(hidden_layer_sizes=(2048,), random_state=42, verbose=True)
gs = GridSearchCV(clf, param_grid, cv=5, n_jobs=-1)
gs.fit(X_train, y_train)

gs.best_estimator_

Iteration 1, loss = 0.14042248
Iteration 2, loss = 0.07742510Iteration 1, loss = 0.15153433
Iteration 1, loss = 0.14827177
Iteration 1, loss = 0.15115151
Iteration 1, loss = 0.15092905
Iteration 1, loss = 0.12866818
Iteration 1, loss = 0.12241432
Iteration 1, loss = 0.12191524
Iteration 1, loss = 0.12288954
Iteration 2, loss = 0.07529893Iteration 2, loss = 0.07243325Iteration 2, loss = 0.07422173Iteration 2, loss = 0.07380420Iteration 2, loss = 0.05828834Iteration 2, loss = 0.07056211Iteration 2, loss = 0.06975872Iteration 2, loss = 0.07109837







Iteration 3, loss = 0.06461544Iteration 3, loss = 0.06193628Iteration 3, loss = 0.06386670Iteration 3, loss = 0.06339668Iteration 3, loss = 0.05066603Iteration 3, loss = 0.06253909Iteration 3, loss = 0.06197439Iteration 3, loss = 0.06300814







Iteration 4, loss = 0.05810027Iteration 4, loss = 0.05551380Iteration 4, loss = 0.05784516Iteration 4, loss = 0.05690456Iteration 4, loss = 0.04525726Iteration 4, loss = 0.05757265Iteration 4, lo

MLPClassifier(activation='relu', alpha=0.0099999999999999985,
       batch_size='auto', beta_1=0.9, beta_2=0.999, early_stopping=False,
       epsilon=1e-08, hidden_layer_sizes=(2048,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=42, shuffle=True,
       solver='adam', tol=0.0001, validation_fraction=0.1, verbose=True,
       warm_start=False)

In [33]:
# Predictiongrid_search
y_pred_train =  gs.best_estimator_.predict(X_train)
y_predict_train_proba = gs.best_estimator_.predict_proba(X_train)[:,0]
        
for i in range(len(y_pred_train)):
    if (y_predict_train_proba[i]<0.9)and(y_predict_train_proba[i]>1-0.9):
        y_pred_train[i]=0

# score
score = compute_pred_score(y_train, y_pred_train)
print('Score sur le train : %s' % score)

Score sur le train : 0.0439962121212


Génération de la prédiction sur le test et enregistrement du fichier à soumettre sur le site:

In [34]:
y_pred = gs.best_estimator_.predict(X_test)
y_predict_proba = gs.best_estimator_.predict_proba(X_test)[:,0]

for i in range(len(y_pred)):
    if (y_predict_proba[i]<0.9)and(y_predict_proba[i]>1-0.9):
        y_pred[i]=0
print np.unique(y_pred)

np.savetxt('y_pred.txt', y_pred, fmt='%d')

[-1  0  1]


Vous pouvez maintenant uploader votre fichier `y_pred.txt` sur le site.

Bonne chance !